In [4]:
%reload_ext autoreload
%autoreload 2

#load libraries
import glob
import os
import pandas as pd
from datetime import datetime
import numpy as np
import main_funcs as mfun

import plot_funcs as pfun
import utils_funcs as utils # utils is from Vape - catcher file: 
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore
import pickle


# Get the data info
pfun.set_figure()

# Get the data info
infoPath = 'C:/Users/Lak Lab/Documents/Github/sideBiasLateralisation/analysis/infoForAnalysis.pkl'
info = pd.read_pickle(infoPath)

## Parameters
fRate = 1000/30
responsiveness_test_duration = 1000.0 #in ms 
pre_frames    = 2000.0# in ms
pre_frames    = int(np.ceil(pre_frames/fRate))
post_frames   = 6000.0 # in ms
post_frames   = int(np.ceil(post_frames/fRate))
analysisWindowDur = 750 # in ms
analysisWindowDur = int(np.ceil(analysisWindowDur/fRate))

In [9]:
info.recordingList

,animalID,recordingDate,recordingID,sessionName,learningData,twoP,path,sessionNameWithPath,analysispathname,filepathname,suite2pPath,imagingDataExtracted,imagingDataPath
0,CMN001,2025-04-09,1,2025-04-09_1_CMN001,False,True,Z:/CMN001\2025-04-09,Z:/CMN001\2025-04-09\1\2025-04-09_1_CMN001_Blo...,C:/Users/Lak Lab/Documents/Github/sideBiasLate...,Z:/CMN001\2025-04-09\1,Z:/CMN001\2025-04-09\TwoP\2025-04-09_t-001\sui...,1.0,C:/Users/Lak Lab/Documents/Github/sideBiasLate...


In [14]:
filenameCSV = info.recordingList.analysispathname[0] + info.recordingList.sessionName[0] + '_CorrectedeventTimes.csv'
filenameCSV = [f for f in glob.glob(filenameCSV)]  
filenameCSV

[]

In [ ]:
#Explore dynamics in each sessions
# For each recording one file saved, each index is a cell 

### THIS IS CODE SNIPPET FOR ONE SESSION!!
#recordingList_index = np.where((info.recordingList['animalID'] == 'OFZ011') & (info.recordingList['recordingDate'] == '2023-07-07'))
#print(recordingList_index)
#ind = recordingList_index[0][0]
pd.set_option('mode.chained_assignment', None)

for ind, recordingDate in enumerate(info.recordingList.recordingDate):

   if (info.recordingList.imagingDataExtracted[ind]==1):
      try:
         print(str(ind) + ': Creating: ' + info.recordingList.analysispathname[ind])
         #Create a huge dictionary with all cells and parameters for each cell
         pathname = info.recordingList.analysispathname[ind]

         ########## Organise stimuli times 
         filenameCSV = info.recordingList.analysispathname[ind] + info.recordingList.sessionName[ind] + '_CorrectedeventTimes.csv'
         filenameCSV = [f for f in glob.glob(filenameCSV)]    
         behData     = pd.read_csv(filenameCSV[0], header=0)
         visTimes    = behData['stimulusOnsetTime']
         rewardTimes = behData['rewardTime']
         choice      = behData['choice']
         choiceTimes = behData['choiceStartTime']
         # Make a variable for rewarded trials
         rewarded    =  behData['rewardTime'].notna() # True for rewarded which also means correct response
         # Calculate reward Time for non-rewarded Trials
         choiceCompleteTimes = behData['choiceCompleteTime']
         diff_time = np.nanmean(rewardTimes -choiceTimes)
         nan_indices = np.isnan(rewardTimes) # unrewarded trials
         rewardTimes[nan_indices] = choiceCompleteTimes[nan_indices] + diff_time

         # Make a variable for stimType - THISS  SECTION NEEDS TO BE CHANGED FOR EACH EXPERIMENT
         stimulusLeftPayoff_1  = behData['stimulusLeftPayoff_1']
         stimulusLeftPayoff_2  = behData['stimulusLeftPayoff_2']
         stimulusRightPayoff_1 = behData['stimulusRightPayoff_1']
         stimulusRightPayoff_2 = behData['stimulusRightPayoff_2']
         stimType = pd.Series([''] * len(behData['choice']))
         stimType [:] = 'Gamble'
         idx = np.sum([stimulusLeftPayoff_1,stimulusLeftPayoff_2,stimulusRightPayoff_1,stimulusRightPayoff_2], axis=0)==6
         stimType [idx] = 'Large Safe'
         idx = (np.sum([stimulusLeftPayoff_1,stimulusLeftPayoff_2,stimulusRightPayoff_1,stimulusRightPayoff_2], axis=0) ==3) & (stimulusLeftPayoff_1 == stimulusLeftPayoff_2) & (stimulusRightPayoff_1 == stimulusRightPayoff_2)
         stimType [idx] = 'Medium Safe'

         # Get the stim start times 
         filenameTXT = os.path.join(info.recordingList.path[ind],'twoP') +'\*_imaging_frames.txt'
         filenameTXT= [f for f in glob.glob(filenameTXT)]    
         frame_clock = pd.read_csv(filenameTXT[0],  header= None)
            
         stimFrameTimes    = utils.stim_start_frame_Dual2Psetup (frame_clock, visTimes)
         rewardFrameTimes  = utils.stim_start_frame_Dual2Psetup (frame_clock, rewardTimes)
         choiceFrameTimes  = utils.stim_start_frame_Dual2Psetup (frame_clock, choiceTimes)

         ########## Organise calcium imaging traces 
         imData = pd.read_pickle (pathname +'imaging-data.pkl')
         fluR      = imData['flu']
         stat      = imData['stat']

         # clean detrended traces
         flu = utils.clean_traces(fluR)
         #flu_normalised = mfun.norm_to_zero_one (flu)
         flu_normalised = zscore (fluR)
         dffTrace_reward ={} 
         dffTrace_mean_reward ={}
         dffTrace_stimuli ={} 
         dffTrace_mean_stimuli ={}
         dffTrace_choice ={}
         dffTrace_mean_choice ={}
         dff_mean_stimuli ={}
         dff_mean_reward ={}
         dff_mean_choice ={}

      ### Get dff values for 2 time windows - CHECK THIS PART
         tTypesName = ['Rewarded', 'Unrewarded',              # reward type
                       'Left choices', 'Right choices',       # choice type
                       'Large Safe','Medium Safe', 'Gamble',  # stimuli type
                       'Rewarded Left','Rewarded Right', 'Unrewarded Left','Unrewarded Right',
                       'Rewarded Large Safe',   'Rewarded Medium Safe' ,    'Rewarded Gamble', 
                       'Unrewarded Large Safe', 'Unrewarded Medium Safe' ,  'Unrewarded Gamble' ]
         tTypes = [(rewarded==True),(rewarded!=True),          # reward type
                  (choice == 'Left'),(choice == 'Right'),      # choice type
                  (stimType =='Large Safe'),  (stimType =='Medium Safe'), (stimType =='Gamble'), # stimuli type
                  ((rewarded == True) & (choice == 'Left')), ((rewarded == True) & (choice == 'Right')),
                  ((rewarded != True) & (choice == 'Left')), ((rewarded != True) & (choice == 'Right')),
                  ((rewarded == True) & (stimType == 'Large Safe')), ((rewarded == True) & (stimType == 'Medium Safe')), ((rewarded == True) & (stimType == 'Gamble')),
                  ((rewarded != True) & (stimType == 'Large Safe')), ((rewarded != True) & (stimType == 'Medium Safe')), ((rewarded != True) & (stimType == 'Gamble')),
         ]


         for indx, t in enumerate(tTypesName) :

            selected_indices         = [rewardFrameTimes[i] for i in np.where((tTypes[indx].values==True)& (~np.isnan(rewardFrameTimes)))[0]]  
            selected_indices         = [value for value in selected_indices if value == value]
            dffTrace_reward[t]       = utils.flu_splitter(flu,selected_indices, pre_frames, post_frames)  # Cell x time x trial
            dffTrace_mean_reward[t]  = np.mean(dffTrace_reward[t],2) if len(selected_indices)>2 else None # Cell x time
            dff_mean_reward[t]       = np.nanmean(dffTrace_mean_reward[t][:, (pre_frames): (pre_frames + analysisWindowDur)],1) if len(selected_indices)>2 else None
                  
            selected_indices         = [stimFrameTimes[i] for i in np.where(tTypes[indx].values==True)[0]]
            selected_indices         = [value for value in selected_indices if value == value]
            dffTrace_stimuli[t]      = utils.flu_splitter(flu, selected_indices, pre_frames, post_frames) # Cell x time x trial
            dffTrace_mean_stimuli[t] = np.mean(dffTrace_stimuli[t],2) if len(selected_indices)>2 else None # Cell x time
            dff_mean_stimuli[t]       = np.nanmean(dffTrace_mean_stimuli[t][:, (pre_frames): (pre_frames + analysisWindowDur)],1) if len(selected_indices)>2 else None

            selected_indices         = [choiceFrameTimes[i] for i in np.where((tTypes[indx].values==True)& (~np.isnan(choiceFrameTimes)))[0]] 
            selected_indices         = [value for value in selected_indices if value == value]
            dffTrace_choice[t]       = utils.flu_splitter(flu, selected_indices, pre_frames, post_frames) # Cell x time x trial
            dffTrace_mean_choice[t]  = np.mean(dffTrace_choice[t],2) if len(selected_indices)>2 else None # Cell x time
            dff_mean_choice[t]       = np.nanmean(dffTrace_mean_choice[t][:, (pre_frames): (pre_frames + analysisWindowDur)],1) if len(selected_indices)>2 else None

         #### Save params
         filenameINFO = pathname + 'imaging-dffTrace.pkl'
         print('Saving: '+ filenameINFO)
         with open(filenameINFO, 'wb') as f:
            pickle.dump([dffTrace_reward,dffTrace_stimuli, dffTrace_choice] , f)
         filenameINFO = pathname + 'imaging-dffTrace_mean.pkl'
         print('Saving: '+ filenameINFO)
         with open(filenameINFO, 'wb') as f:
            pickle.dump([dffTrace_mean_reward,
                         dffTrace_mean_stimuli,
                         dffTrace_mean_choice ] , f)
         print('Saved successfully: '+ filenameINFO)
         filenameINFO = pathname + 'imaging-dff_mean.pkl'
         print('Saving: '+ filenameINFO)
         with open(filenameINFO, 'wb') as f:
            pickle.dump([dff_mean_reward,
                         dff_mean_stimuli,
                         dff_mean_choice ] , f)
         print('Saved successfully: '+ filenameINFO)


         ### Plot dynamics:# Explore reward, stimuli, choice representation
      
         pfun.set_figure()
         figsPath = info.figsPath
         selectedSession = 0
         duration = '3sec'
         colormap = None

         # Heat Plots
         savefigname = 'SessionComparison-heatmap-reward_' + duration
         analysis_params = ['Rewarded', 'Unrewarded']
         pfun.heatmap_sessions(dffTrace_mean_reward,analysis_params, colormap,
                              selectedSession, duration, savefigname, pathname) 
         plt.close()
                        
         savefigname = 'SessionComparison-heatmap-choice_' + duration
         analysis_params = ['Left choices','Right choices']
         pfun.heatmap_sessions(dffTrace_mean_choice,analysis_params, colormap,
                              selectedSession, duration, savefigname, pathname) 
         plt.close()

         savefigname = 'SessionComparison-heatmap-stimuli_' + duration
         analysis_params = ['Large Safe','Medium Safe', 'Gamble']
         pfun.heatmap_sessions(dffTrace_mean_stimuli,analysis_params, colormap,
                              selectedSession, duration, savefigname, pathname) 
         plt.close()

         # Mean Plots 
         colormap = 'Set2'

         savefigname = 'SessionComparison-mean-reward_' + duration
         analysis_params = ['Rewarded', 'Unrewarded']
         pfun.lineplot_sessions(dffTrace_mean_reward,analysis_params, colormap,
                              selectedSession, duration, savefigname, pathname) 
         plt.close()
         
         savefigname = 'SessionComparison-mean-stimuli_' + duration
         analysis_params = ['Large Safe','Medium Safe', 'Gamble']
         pfun.lineplot_sessions(dffTrace_mean_stimuli,analysis_params, colormap,
                              selectedSession, duration, savefigname, pathname)
         plt.close()

         savefigname = 'SessionComparison-mean-choice_' + duration
         analysis_params = ['Left choices','Right choices']
         pfun.lineplot_sessions(dffTrace_mean_choice,analysis_params, colormap,
                              selectedSession, duration, savefigname, pathname) 
         plt.close()
         print('Completed')
         info.recordingList.loc[ind,'analysisVariableExtracted'] = 0 
      except:
         info.recordingList.loc[ind,'analysisVariableExtracted'] = 1
         print('Problem in plotting')
